In [ ]:
spfs = hf.system.spfs;
C = hf.C;
C_tilde = C';
l = hf.system.l
h = hf.system.h
u = hf.system.u;

In [ ]:
function transform_spfs(spfs, C)
    spfs2 = zero.(spfs)
    l = length(spfs)
    
    for i in 1:l
        psi1 = spfs2[i]
        for j in 1:l
            psi1 .+= C[j, i] .* spfs[j]
        end
    end
    return spfs2
end

In [ ]:
@time spfs2 = transform_spfs(spfs, C);

In [ ]:
function transform_onebody(h, C)
    h2 = zero(h)
    l = size(C)[1]
    
    @inbounds for a in 1:l
        for b in 1:l
            for i in 1:l
                for j in 1:l
                    h2[a, b] += C[i, a] * C[j, b] * h[i, j]
                end
            end
        end
    end
    return h2
end

In [ ]:
function transform_onebody2(h, C)
    return C' * h * C;
end

In [ ]:
@time h2 = transform_onebody(h, C);

In [ ]:
@time h3 = transform_onebody2(h, C);

In [ ]:
h2 ≈ h3

## Twobody

In [ ]:
function transform_twobody_slow(u, C)
    u2 = zero(u)
    l = length(C[:,1])
    
    for a in 1:l, b in 1:l, c in 1:l, d in 1:l
        for i in 1:l, j in 1:l, k in 1:l, m in 1:l
            @inbounds u2[a, b, c, d] += C[i, a] * C[j, b] * C[k, c] * C[m, d] * u[i, j, k, m]
        end
    end
    return u2
end

In [ ]:
@time u_slow = transform_twobody_slow(u, C);

In [ ]:
function transform_twobody(u, C)
    u2 = zero(u)
    u3 = zero(u)
    l = size(C)[1]
    
    @inbounds for a in 1:l, b in 1:l, c in 1:l
        for i in 1:l
            for j in 1:l
                u2[a, b, c, i] += C[j, i] * u[a, b, c, j]
            end
        end
    end
    
    @inbounds for a in 1:l, b in 1:l, d in 1:l
        for i in 1:l
            for j in 1:l
                u3[a, b, i, d] += C[j, i] * u2[a, b, j, d]
            end
        end
    end
    
    @inbounds for a in 1:l, c in 1:l, d in 1:l
        for i in 1:l
            u2[a, i, c, d] = 0
            for j in 1:l
                u2[a, i, c, d] += C[j, i] * u3[a, j, c, d]
            end
        end
    end
    
    @inbounds for b in 1:l, c in 1:l, d in 1:l
        for i in 1:l
            u3[i, b, c, d] = 0
            for j in 1:l
                u3[i, b, c, d] += C[j, i] * u2[j, b, c, d]
            end
        end
    end
    
    return u3
end

In [ ]:
@time u2 = transform_twobody(u, C);

In [ ]:
u2 ≈ u_py

In [ ]:
using PyCall

In [ ]:
using TensorOperations

In [ ]:
function transform_twobody(u, C)
    @tensor begin
        u2[a, b, c, i] := u[a, b, c, d] * C[d, i]
        u3[a, b, i, d] := u2[a, b, c, d] * C[c, i]
        u2[a, i, c, d] = u3[a, b, c, d] * C[b, i]
        u3[i, b, c, d] = u2[a, b, c, d] * C[a, i]
    end
    return u3
end

In [ ]:
u3 = transform_twobody(u, C);

In [ ]:
println(fasit ≈ u3)

In [ ]:
py"""
import numpy as np

def please(u, C, C_tilde):
    _u = np.tensordot(u, C, axes=(3, 0))
    _u = np.tensordot(_u, C, axes=(2, 0)).transpose(0, 1, 3, 2)
    _u = np.tensordot(_u, C_tilde, axes=(1, 1)).transpose(0, 3, 1, 2)
    _u = np.tensordot(C_tilde, _u, axes=(1, 0))
    return _u
"""
@time fasit = py"please"(u, C, C');

In [ ]:
findmax(fasit), findmin(fasit)